In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display	
from IPython.display import display, clear_output
import pandas as pd

import ift6758.client.serving_client as sc
import ift6758.client.game_client as gc

g_client = gc.GameClient(ip = 'serving_app', port = 3000)
client = sc.ServingClient(ip ='serving_app', port = 3000)
client.logs()

In [20]:
class AE:
  def __init__(self):
    self.df = pd.DataFrame(columns=['gameID','home name','away name','team name','period time','period num','coordinates x','coordinates y','Distance from net','Angle from net','shot type','Time from the last event','last event type','last coordinates x','last coordinates y','Distance from the last event','Rebound','Change in shot angle','Speed','Is goal','Goal probability','home Goal','away Goal'])
  
  def add_gameID(self, gameID):
    self.df.iloc[self.df.shape[0]-1,0] = str(gameID)
  
  def find_gameID(self, gameID):
    return self.df[self.df['gameID'] == gameID]
  
  def gameID_list(self, gameID):
    return list(self.df.drop_duplicates()['gameID'])

  def gameID_exist(self, gameID):
    id_list = AE.gameID_list(self, gameID)
    return gameID in id_list
  def init_game_ID(self):
    self.df = pd.DataFrame(columns=['gameID','home name','away name','team name','period time','period num','coordinates x','coordinates y','Distance from net','Angle from net','shot type','Time from the last event','last event type','last coordinates x','last coordinates y','Distance from the last event','Rebound','Change in shot angle','Speed','Is goal','Goal probability','home Goal','away Goal']) 
all_events_df = AE()


button = widgets.Button(description="Ping Game")
output = widgets.Output()
temp_model = ''
def show(workspace, model, version):
  client.download_registry_model(workspace, model, version)
  global temp_model
  if temp_model != model:
    all_events_df.init_game_ID()
  temp_model = model

interact(show, workspace = ['maskedviper'], model = ['xgb', 'random_forest'], version = ['1.0.0']);

text = widgets.Text(
    value='2016020001',
    description='GameID',
    disabled=False
)
display(text)

def callback(gameID):
  if all_events_df.gameID_exist(gameID.value):
    idx = all_events_df.df[all_events_df.df['gameID'] == gameID.value].shape[0]
  else:
    idx = 0
  event,_,past_events = g_client.ping_game(gameID.value, idx, all_events_df.df)
  #event,_,past_events = ping_game(gameID.value, idx, all_events_df.df)
  if event.shape[0] == 0:
    event = past_events.loc[past_events.shape[0]-1]
    left_time, period_num = g_client.get_num_and_left_time(event)
    home, away = g_client.get_team_name(event)
    hp, ap = g_client.each_goal_prob(event)
    line = '|' + '-' * 71 + '|'
    team_headers = '|' + 'Home Team'.center(34) + ' | ' + 'Away Team'.center(34) + '|'
    period_line = '|' + ('Period:' + str(int(period_num)) + '  --  ' +  str(left_time)).center(71) + '|'
    team_names = '|' + home.center(34) + ' | ' + away.center(34) + '|'
    expected_goals = '|' + str(hp).center(34) + ' | ' + str(ap).center(34) + '|'

    print(line)
    print(period_line)
    print(line)
    print(team_headers)
    print(line)
    print(team_names)
    print(line)
    print(expected_goals)
    print(line)
    display_df = all_events_df.df[all_events_df.df['gameID'] == gameID.value]
    display(display_df.reset_index(drop=True).iloc[:,:-2])
    return ""
  #all_events_df.add_gameID(gameID.value)

  left_time, period_num = g_client.get_num_and_left_time(event)
  home, away = g_client.get_team_name(event)
  hp, ap = g_client.each_goal_prob(event)
  line = '|' + '-' * 71 + '|'
  team_headers = '|' + 'Home Team'.center(34) + ' | ' + 'Away Team'.center(34) + '|'
  period_line = '|' + ('Period:' + str(int(period_num)) + '  --  ' +  str(left_time)).center(71) + '|'
  team_names = '|' + home.center(34) + ' | ' + away.center(34) + '|'
  expected_goals = '|' + str(hp).center(34) + ' | ' + str(ap).center(34) + '|'
  print(line)
  print(period_line)
  print(line) 
  print(team_headers)
  print(line)
  print(team_names)
  print(line)
  print(expected_goals)
  print(line)
  display_df = all_events_df.df[all_events_df.df['gameID'] == gameID.value]
  display(display_df.reset_index(drop=True).iloc[:,:-2])

text.on_submit(callback)


#index = 0
display(button, output)
def on_button_clicked(b):
  with output:
    clear_output()
    callback(gameID=text)
button.on_click(on_button_clicked)

interactive(children=(Dropdown(description='workspace', options=('maskedviper',), value='maskedviper'), Dropdo…

Text(value='2016020001', description='GameID')

Button(description='Ping Game', style=ButtonStyle())

Output()